In [ ]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import special_ortho_group
from hisel.select import HSICSelector as Selector

# Sin transform 

In [ ]:
dim_x = 10
dim_y = 1
dim_z = 1

batch_size = int(1e+4)
minibatch_size = 250
num_of_samples = int(1e+4)
number_of_epochs = 3

In [ ]:
transform_tilde = np.eye(dim_z)[:dim_y]
A = np.random.permutation(np.concatenate((np.eye(dim_z), np.zeros((dim_z, dim_x - dim_z))), axis=1).T).T
transform = transform_tilde @ A

In [ ]:
x_samples = np.random.uniform(size=(num_of_samples, dim_x))
tt = np.repeat(np.expand_dims(transform, axis=0), repeats=num_of_samples, axis=0)
prey = (tt @ np.expand_dims(x_samples, axis=2))[:, :, 0]
y_samples = np.random.normal(0, 3e-1, size=prey.shape) 
y_samples[:, 0] += np.sin(2*np.pi*prey[:, 0])

### Viz of relations between target and features

Relation between $y$ and the correct feature

In [ ]:
expected_features = np.argsort(np.sum(A, axis=0))[::-1][:dim_z]
sns.scatterplot(x=x_samples[:, expected_features[0]], y=y_samples[:, 0])

Relation between $y$ and a wrong feature

In [ ]:
nonrelevant = set(range(dim_x)).difference(set(expected_features))
featureidx = np.random.choice(list(nonrelevant))
sns.scatterplot(x=x_samples[:, featureidx], y=y_samples[:, 0])

In [ ]:
projector = Selector(x_samples, y_samples)

In [ ]:
curve = projector.regularization_curve(
    batch_size=batch_size,
    minibatch_size=minibatch_size,
    number_of_epochs=number_of_epochs
)
paths = projector.lasso_path()

#### Sorted features by decreasing importance

In [ ]:
print(f'Sorted features by decreasing importance: {projector.ordered_features}')

### Test selection

In [ ]:
expected_features = np.argsort(np.sum(A, axis=0))[::-1][:dim_z]
noise_features = set(range(dim_x)).difference(set(expected_features))
selected_features = np.argsort(paths.iloc[-1, :])[::-1][:dim_z]
print(f'Expected features: {sorted(list(expected_features))}')
print(f'Selected features: {sorted(list(selected_features))}')

## Comparison with sklearn

In [ ]:
from sklearn.feature_selection import f_regression, mutual_info_regression

In [ ]:
fstats, _ = f_regression(x_samples, np.linalg.norm(y_samples, axis=1))
fstats /= np.max(fstats)
f_selection = np.argmax(fstats)
print(f'f_selection: {f_selection}')

In [ ]:
mi = mutual_info_regression(x_samples, np.linalg.norm(y_samples, axis=1))
mi /= np.max(mi)
mi_selection = np.argmax(mi)
print(f'mi_selection: {mi_selection}')

# Linear and non-linear  transformation in high dimension

In [ ]:
dim_x = 20
dim_y = 3
dim_z = 5

batch_size = int(1e+4)
minibatch_size = 250
num_of_samples = int(1e+4)
number_of_epochs = 3

In [ ]:
transform_tilde = special_ortho_group.rvs(dim_z)[:dim_y]
A = np.random.permutation(np.concatenate((np.eye(dim_z), np.zeros((dim_z, dim_x - dim_z))), axis=1).T).T
transform = transform_tilde @ A

In [ ]:
x_samples = np.random.uniform(size=(num_of_samples, dim_x))
tt = np.repeat(np.expand_dims(transform, axis=0), repeats=num_of_samples, axis=0)
prey = (tt @ np.expand_dims(x_samples, axis=2))[:, :, 0]
y_samples = np.random.normal(0, 1e-2, size=prey.shape)  # np.zeros_like(prey)
y_samples[:, 0] = np.sin(2*np.pi*prey[:, 0])
y_samples[:, 1] = np.cos(2*np.pi*prey[:, 1])
y_samples[:, 2] = np.sin(2*np.pi*prey[:, 2])

### Viz of relations between target and features

Becasue of the initial rotation, visual inspection of the relation between features and target does not give insights on which features should be selected

Relation between $y$ and a correct feature

In [ ]:
expected_features = np.argsort(np.sum(A, axis=0))[::-1][:dim_z]
sns.scatterplot(x=x_samples[:, expected_features[0]], y=y_samples[:, 0])

Relation between $y$ and a wrong feature

In [ ]:
nonrelevant = set(range(dim_x)).difference(set(expected_features))
featureidx = np.random.choice(list(nonrelevant))
sns.scatterplot(x=x_samples[:, featureidx], y=y_samples[:, 0])

In [ ]:
projector = Selector(x_samples, y_samples)

In [ ]:
curve = projector.regularization_curve(
    batch_size=batch_size,
    minibatch_size=minibatch_size,
    number_of_epochs=number_of_epochs
)

In [ ]:
paths = projector.lasso_path()

#### Sorted features by decreasing importance

In [ ]:
print(f'Sorted features by decreasing importance: {projector.ordered_features}')

### Test selection

In [ ]:
expected_features = np.argsort(np.sum(A, axis=0))[::-1][:dim_z]
noise_features = set(range(dim_x)).difference(set(expected_features))
selected_features = np.argsort(paths.iloc[-1, :])[::-1][:dim_z]
print(f'Expected features: {sorted(list(expected_features))}')
print(f'Selected features: {sorted(list(selected_features))}')

## Comparison with sklearn

In [ ]:
from sklearn.feature_selection import f_regression, mutual_info_regression

In [ ]:
fstats, _ = f_regression(x_samples, np.linalg.norm(y_samples, axis=1))
fstats /= np.max(fstats)
f_selection = np.argsort(fstats)[::-1][:dim_z]
print(f'f_selection: {sorted(f_selection)}')

In [ ]:
mi = mutual_info_regression(x_samples, np.linalg.norm(y_samples, axis=1))
mi /= np.max(mi)
mi_selection = np.argsort(mi)[::-1][:dim_z]
print(f'mutual information: {mi[mi_selection]}')
print(f'mi_selection: {sorted(mi_selection)}')

## Regularisation curve

#### Cumulative beta

In [ ]:
plt.plot(np.arange(1, 1+len(curve)), curve)

#### Absolute beta

In [ ]:
plt.plot(np.arange(1, len(curve)), np.abs(np.diff(curve)))

## Lasso paths

In [ ]:
paths.plot(figsize=(10, 5))